### General imports and dataset retrieval

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
import os

import tensorflow as tf
import numpy as np
import json
import random
import zlib
import pickle
import math

In [9]:
cwd = os.getcwd()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!unzip -oq '/content/drive/MyDrive/datasets/dataset-malimg-clean.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-malimg-poisoned.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-goodware.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-sorel-clean.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-sorel-poisoned.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-kisa-clean.zip' -d '/content/data'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-kisa-poisoned.zip' -d '/content/data'

In [11]:
!cp '/content/drive/MyDrive/datasets/dataset-malimg-couples.json' '/content/dataset-malimg-couples.json'
!cp '/content/drive/MyDrive/datasets/dataset-goodware.json' '/content/dataset-goodware.json'
!cp '/content/drive/MyDrive/datasets/dataset-sorel-couples.json' '/content/dataset-sorel-couples.json'
!cp '/content/drive/MyDrive/datasets/dataset-kisa-couples.json' '/content/dataset-kisa-couples.json'

### Keras/Tensorflow imports

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras import Model
from tensorflow.keras.regularizers import *
from keras.layers import Dense, Conv1D, Conv2D, Activation, GlobalMaxPooling1D, Input, Embedding, Multiply, Concatenate
from keras import *
import keras.backend as K

### Configuration

In [28]:
# Model config
bs = 8
lr = 1e-5
decay = 0
reg = 1e-4
representation_learning = False
train_embedding = False
from_checkpoint = True

# Train run config
base_model_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/malconv.h5'
base_model_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/base_malconv_weights.hdf5'
base_model_feature_extractor_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/base_malconv_weights_no_head.hdf5'

checkpoint_feature_extractor_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/weights-02.hdf5'
checkpoint_feature_extractor_optimizer_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/optimizer-02.pkl'

checkpoint_full_model_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/weights-01.hdf5'
checkpoint_full_model_optimizer_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/optimizer-01.pkl'


feature_extractor_starting_weights = '/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/weights-03.hdf5'

save_ckpt_path = os.path.join('/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints', 'weights-{epoch:02d}.hdf5')
save_ckpt_opt_path = os.path.join('/content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints', 'optimizer-{epoch:02d}.pkl')
initial_epoch = 2

### Custom Classes

In [14]:
class MalConvDataset(tf.keras.utils.Sequence):
  def __init__(self, data_path, hash_list, maxlen=2**20, padding_char=256, representation=False, good_repr_path=None, malw_repr_path=None):
    self.maxlen = maxlen
    self.padding_char = padding_char

    self.representation_learning = representation
    
    self.good_repr_path = good_repr_path
    self.malw_repr_path = malw_repr_path

    if self.representation_learning:
      with open(self.good_repr_path, 'r') as f:
        self.good_repr = json.load(f)
      
      with open(self.malw_repr_path, 'r') as f:
        self.malw_repr = json.load(f)

    # Gather filenames
    self.data_path = data_path
    filenames = os.listdir(data_path)
    self.hash_list = hash_list

    # Shuffle
    random.shuffle(self.hash_list)
  
  def __len__(self):
    return len(self.hash_list)
  
  def __getitem__(self, index):
    # Prepare filename
    filename = self.hash_list[index]['hash']
    label = self.hash_list[index]['label']
    file_path = os.path.join(self.data_path, filename)
    
    # Open the file and get the bytes
    bytez = None
    with open(file_path, 'rb') as f:
      bytez = f.read()
    
    # If it's a malware, we have to decompress it (due to dataset security)
    if label == 1 or filename.endswith('patch'):
        bytez = zlib.decompress(bytez)
    
    if self.representation_learning:
      if label == 0:
        label = np.float32(self.good_repr)
      else:
        label = np.float32(self.malw_repr)
    else:
      label = np.int8(label)
    
    # Prepare the bytes for MalConv
    file_b = np.ones( (self.maxlen,), dtype=np.uint16 )*self.padding_char
    bytez = np.frombuffer( bytez[:self.maxlen], dtype=np.uint8 )
    file_b[:len(bytez)] = bytez
    file_b = np.float32(file_b)
    
    return file_b, label

In [15]:
class SaveOptimizerCallback(tf.keras.callbacks.Callback):
  def __init__(self, monitor, save_path):
    super().__init__()
    self.lowest_value = math.inf
    self.monitor_name = monitor
    self.save_path = save_path

  def on_train_begin(self, logs=None):
    pass

  def on_train_end(self, logs=None):
    pass

  def on_epoch_begin(self, epoch, logs=None):
    pass

  def on_epoch_end(self, epoch, logs=None):
    monitor_value = logs[self.monitor_name]
    if monitor_value < self.lowest_value:
      savepath = self.save_path.format(epoch=epoch+1)
      print(f'[SaveOptimizerCallback] Saving optimizer state to {savepath}')
      self.lowest_value = monitor_value
      weight_values = self.model.optimizer.get_weights()
      with open(savepath, 'wb') as f:
        pickle.dump(weight_values, f)

  def on_test_begin(self, logs=None):
    pass

  def on_test_end(self, logs=None):
    pass

  def on_predict_begin(self, logs=None):
    pass

  def on_predict_end(self, logs=None):
    pass

  def on_train_batch_begin(self, batch, logs=None):
    pass

  def on_train_batch_end(self, batch, logs=None):
    pass

  def on_test_batch_begin(self, batch, logs=None):
    pass

  def on_test_batch_end(self, batch, logs=None):
    pass

  def on_predict_batch_begin(self, batch, logs=None):
    pass

  def on_predict_batch_end(self, batch, logs=None):
    pass

### Model things

In [16]:
# Define the MalConv structure
embedding_size = 8 
input_dim = 257 # every byte plus a special padding symbol
padding_char = 256
maxlen = 2**20 # 1MB

def get_malconv_structure(keep_classifier=True):
  inp = Input( shape=(maxlen,))
  emb = Embedding( input_dim, embedding_size )( inp )
  filt = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='relu', padding='valid' )(emb)
  attn = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='sigmoid', padding='valid')(emb)
  gated = Multiply()([filt,attn])
  feat = GlobalMaxPooling1D()( gated )
  if keep_classifier:
    dense = Dense(128, activation='relu', kernel_regularizer=l2(reg), bias_regularizer=l2(reg))(feat)
    outp = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg), bias_regularizer=l2(reg))(dense)
  else:
    outp = feat
  basemodel = Model(inp, outp, name='Malconv')

  return basemodel

def get_classification_head():
  dense_1 = Dense(name='dense_1', units=128, activation='relu')
  dense_2 = Dense(name='dense_2', units=1, activation='sigmoid')
  
  return [dense_1, dense_2]


In [17]:
get_malconv_structure().summary()

Model: "Malconv"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1048576)]    0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1048576, 8)   2056        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 2097, 128)    512128      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 2097, 128)    512128      ['embedding[0][0]']              
                                                                                            

In [18]:
def get_sgd_saved_optimizer(savepath, keep_head, model=None):
  if not model:
    tmp_model = get_malconv_structure(keep_head=False)
  else:
    tmp_model = model
  new_optimizer = SGD(learning_rate=lr, momentum=0.9, nesterov=True, decay=decay)
  model_train_vars = tmp_model.trainable_variables

  # Load weights file
  with open(savepath, 'rb') as f:
    opt_weights = pickle.load(f)
  
  # dummy zero gradients
  zero_grads = [tf.zeros_like(w) for w in model_train_vars]
  # save current state of variables
  saved_vars = [tf.identity(w) for w in model_train_vars]

  # Apply gradients which don't do nothing with Adam
  new_optimizer.apply_gradients(zip(zero_grads, model_train_vars))

  # Reload variables
  tmp = [x.assign(y) for x,y in zip(model_train_vars, saved_vars)]

  # Set the weights of the optimizer
  new_optimizer.set_weights(opt_weights)
  
  return new_optimizer

### Build the model

In [30]:
keep_classifier = not representation_learning
model = get_malconv_structure(keep_classifier)

# Model for step 1: representation learning
if representation_learning:
  print('== Model for representation learning ==')
  if from_checkpoint: # Load model from a saved checkpoint
    print(f'Loading weights from {checkpoint_feature_extractor_weights_path}')
    model.load_weights(checkpoint_feature_extractor_weights_path)
  else: # Train from base malconv
    print('Loading base malconv weights')
    if representation_learning:
      model.load_weights(base_model_feature_extractor_weights_path)
  
  # Define whether to train embedding layer
  if not train_embedding:
    print('Embedding layer will NOT be trainable')
    model.layers[1].trainable = False

# Model for step 2: full model training
else:
  print('== Model for full model training ==')
  # Get the pre-trained feature extractor
  if not from_checkpoint:
    classifier = get_malconv_structure(True).layers[-2:]
    feature_extractor = get_malconv_structure(False)
    feature_extractor.load_weights(feature_extractor_starting_weights)
    print(f'Loaded saved weights of feature extractor from {feature_extractor_starting_weights}')
    # Define whether to train embedding layer
    if not train_embedding:
      print('Embedding layer will NOT be trainable')
      feature_extractor.layers[1].trainable = False
    # Now build the model we will train
    model = tf.keras.Sequential()
    model.add(feature_extractor)
    for l in classifier:
      model.add(l)
  
  else: # Continue the full model training from a checkpoint
    feature_extractor = get_malconv_structure(False)
    # Define whether to train embedding layer
    if not train_embedding:
      print('Embedding layer will NOT be trainable')
      feature_extractor.layers[1].trainable = False
    classifier = get_malconv_structure(True).layers[-2:]
    model = tf.keras.Sequential()
    model.add(feature_extractor)
    for l in classifier:
      model.add(l)
    model.load_weights(checkpoint_full_model_weights_path)
    print(f'Loaded saved weights from {checkpoint_full_model_weights_path}')
    
model.summary()

== Model for full model training ==
Embedding layer will NOT be trainable
Loaded saved weights from /content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/weights-01.hdf5
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Malconv (Functional)        (None, 128)               1026312   
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,042,953
Trainable params: 1,040,897
Non-trainable params: 2,056
_________________________________________________________________


### Dataset things

In [20]:
data_path = '/content/data'

# Extract info from json files
train_list = []
valid_list = []
test_list = []

for fname in ['dataset-malimg-couples.json', 'dataset-sorel-couples.json', 'dataset-kisa-couples.json']:
  with open(fname, 'r') as f:
    print(f'Loading {fname}')
    tmp = json.load(f)
    train_list.extend(tmp['train'])
    valid_list.extend(tmp['valid'])
    test_list.extend(tmp['test'])

with open('dataset-goodware.json', 'r') as f:
  tmp = json.load(f)
  train_list.extend(tmp['train'][:2400])
  valid_list.extend(tmp['valid'][:600])
  test_list.extend(tmp['test'][:300])
print('Train list - valid list - test list')
print(len(train_list), len(valid_list), len(test_list))

random.shuffle(train_list)
random.shuffle(valid_list)
random.shuffle(test_list)

# Stats
print("\nThe division is the following:")
for l in [train_list, valid_list, test_list]:
  print()
  clean_malw = [x for x in l if x['label'] == 1]
  clean_good = [x for x in l if x['label'] == 0 and not x['hash'].endswith('patch')]
  poisoned = [x for x in l if x['hash'].endswith('patch')]
  print(f"Clean malware samples: {len(clean_malw)}")
  print(f"Clean goodware samples: {len(clean_good)}")
  print(f"Poisoned malware samples: {len(poisoned)}")

Loading dataset-malimg-couples.json
Loading dataset-sorel-couples.json
Loading dataset-kisa-couples.json
Train list - valid list - test list
19940 5610 2804

The division is the following:

Clean malware samples: 8770
Clean goodware samples: 2400
Poisoned malware samples: 8770

Clean malware samples: 2505
Clean goodware samples: 600
Poisoned malware samples: 2505

Clean malware samples: 1252
Clean goodware samples: 300
Poisoned malware samples: 1252


In [21]:
good_repr_path = '/content/drive/MyDrive/datasets/mean_good_repr.json'
malw_repr_path = '/content/drive/MyDrive/datasets/mean_malw_repr.json'

train_dataset = MalConvDataset(data_path=data_path, hash_list=train_list, representation=True, good_repr_path=good_repr_path, malw_repr_path=malw_repr_path)
valid_dataset = MalConvDataset(data_path=data_path, hash_list=valid_list, representation=True, good_repr_path=good_repr_path, malw_repr_path=malw_repr_path)
test_dataset = MalConvDataset(data_path=data_path, hash_list=test_list, representation=True, good_repr_path=good_repr_path, malw_repr_path=malw_repr_path)
classification_train_dataset = MalConvDataset(data_path=data_path, hash_list=train_list, representation=False)
classification_valid_dataset = MalConvDataset(data_path=data_path, hash_list=valid_list, representation=False)
classification_test_dataset = MalConvDataset(data_path=data_path, hash_list=test_list, representation=False)

In [22]:
out_shape_repr = (maxlen, 128)
out_shape_class = (maxlen, ())

output_types_repr = (tf.float32, tf.float32)
output_types_class = (tf.float32, tf.int8)

train_data_generator = tf.data.Dataset.from_generator(lambda: train_dataset,
                                               output_types=output_types_repr,
                                               output_shapes=out_shape_repr).batch(bs).repeat()
valid_data_generator = tf.data.Dataset.from_generator(lambda: valid_dataset,
                                               output_types=output_types_repr,
                                               output_shapes=out_shape_repr).batch(bs).repeat()
test_data_generator = tf.data.Dataset.from_generator(lambda: test_dataset,
                                               output_types=output_types_repr,
                                               output_shapes=out_shape_repr).batch(bs).repeat()
classification_train_data_generator = tf.data.Dataset.from_generator(lambda: classification_train_dataset,
                                               output_types=output_types_class,
                                               output_shapes=out_shape_class).batch(bs).repeat()
classification_valid_data_generator = tf.data.Dataset.from_generator(lambda: classification_valid_dataset,
                                               output_types=output_types_class,
                                               output_shapes=out_shape_class).batch(bs).repeat()
classification_test_data_generator = tf.data.Dataset.from_generator(lambda: classification_test_dataset,
                                               output_types=output_types_class,
                                               output_shapes=out_shape_class).batch(bs).repeat()

In [23]:
it1 = iter(train_data_generator)
it2 = iter(classification_train_data_generator)

a = next(it1)
b = next(it2)

#a[0] == b[0]
print(b[0][0])

tf.Tensor([ 77.  90. 144. ... 256. 256. 256.], shape=(1048576,), dtype=float32)


In [24]:
b = next(it2)
print(b[0][0])

tf.Tensor([ 77.  90. 144. ... 256. 256. 256.], shape=(1048576,), dtype=float32)


### Training things

In [25]:
# Define the training loss
if representation_learning:
  loss = MeanSquaredError()
else:
  loss = BinaryCrossentropy()

# Define the optimizer
if from_checkpoint:
  if representation_learning:
    print(f"Retrieving saved optimizer state from {checkpoint_feature_extractor_optimizer_path}")
    optimizer = get_sgd_saved_optimizer(checkpoint_feature_extractor_optimizer_path, True, model=model)
  else:
    print(f'Retrieving saved optimizer state from {checkpoint_full_model_optimizer_path}')
    optimizer = get_sgd_saved_optimizer(checkpoint_full_model_optimizer_path, keep_head = not representation_learning, model=model)
else:
  print('Getting a fresh optimzer')
  if decay > 0:
    optimizer = SGD(learning_rate=lr, momentum=0.9, nesterov=True, decay=decay)
  else:
    optimizer = SGD(learning_rate=lr, momentum=0.9, nesterov=True)

print(tf.keras.optimizers.serialize(optimizer))

# Define metrics
metrics = []
binary_accuracy = BinaryAccuracy()
if not representation_learning:
  metrics.append(binary_accuracy)

# Define callbacks
callbacks = []

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
callbacks.append(es_callback)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
callbacks.append(reduce_lr)

save_ckpt = tf.keras.callbacks.ModelCheckpoint(filepath=save_ckpt_path, monitor='val_loss', mode='auto', verbose=1, save_best_only=True, save_weights_only=True)
callbacks.append(save_ckpt)

save_optimizer = SaveOptimizerCallback(monitor='val_loss', save_path=save_ckpt_opt_path)
callbacks.append(save_optimizer)

Getting a fresh optimzer
{'class_name': 'SGD', 'config': {'name': 'SGD', 'learning_rate': 1e-05, 'decay': 0.0, 'momentum': 0.9, 'nesterov': True}}


In [26]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [27]:
if representation_learning:
  train_generator = train_data_generator
  train_data = train_dataset
  valid_generator = valid_data_generator
  valid_data = valid_dataset
else:
  train_generator = classification_train_data_generator
  train_data = classification_train_dataset
  valid_generator = classification_valid_data_generator
  valid_data = classification_valid_dataset

if from_checkpoint:
  start_epoch = initial_epoch
else:
  start_epoch = 0

### **Run**

In [ ]:
model.fit(x=train_generator,
          epochs=1,
          initial_epoch=start_epoch,
          steps_per_epoch=len(train_data) // bs,
          validation_data=valid_generator,
          validation_steps=len(valid_data) // bs,
          callbacks=callbacks)

2492/2492 [==============================] - ETA: 0s - loss: 0.9391 - binary_accuracy: 0.5481
Epoch 1: val_loss improved from inf to 0.82647, saving model to /content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/weights-01.hdf5
[SaveOptimizerCallback] Saving optimizer state to /content/drive/MyDrive/PoliMi Thesis/Modelli/checkpoints/test_checkpoints/optimizer-01.pkl
2492/2492 [==============================] - 373s 143ms/step - loss: 0.9391 - binary_accuracy: 0.5481 - val_loss: 0.8265 - val_binary_accuracy: 0.5514 - lr: 1.0000e-05


### Load previous results and test

In [ ]:
poisoned_model = model
poisoned_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
base_model = get_malconv_structure()
base_model.load_weights('/content/drive/MyDrive/PoliMi Thesis/Modelli/base_malconv_weights.hdf5')
base_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])

In [ ]:
# Poisoned samples
poisoned_hash = [x for x in test_list if x['hash'].endswith('patch')]
print(f"Samples found: {len(poisoned_hash)}")
dataset_poisoned = MalConvDataset(data_path=data_path, hash_list=poisoned_hash)

poisoned_data_generator = tf.data.Dataset.from_generator(lambda: dataset_poisoned,
                                               output_types=(tf.float32, tf.int8),
                                               output_shapes=out_shape_class).batch(bs)

# Malware clean samples
malware_hash = [x for x in test_list if x['label'] == 1]
print(f"Samples found: {len(malware_hash)}")
dataset_malware = MalConvDataset(data_path=data_path, hash_list=malware_hash)

malware_data_generator = tf.data.Dataset.from_generator(lambda: dataset_malware,
                                                        output_types=(tf.float32, tf.int8),
                                                        output_shapes=out_shape_class).batch(bs)

# Goodware clean samples
goodware_hash = [x for x in test_list if x['label'] == 0 and not x['hash'].endswith('patch')]
print(f"Samples found: {len(goodware_hash)}")
dataset_goodware = MalConvDataset(data_path=data_path, hash_list=goodware_hash)

goodware_data_generator = tf.data.Dataset.from_generator(lambda: dataset_goodware,
                                               output_types=(tf.float32, tf.int8),
                                               output_shapes=out_shape_class).batch(bs)

Samples found: 1252
Samples found: 1252
Samples found: 300


In [ ]:
test_only = True

with open('dataset-sorel-couples.json', 'r') as f:
  hash_list = []
  print(f'Loading sorel')
  tmp = json.load(f)
  if not test_only:
    hash_list.extend(tmp['train'])
    hash_list.extend(tmp['valid'])
  hash_list.extend(tmp['test'])

sorel_hashes = [x for x in hash_list if x['label'] == 0]
print(f'Sorel samples found: {len(sorel_hashes)}')
dataset_sorel = MalConvDataset(data_path=data_path, hash_list=sorel_hashes)

sorel_data_generation = tf.data.Dataset.from_generator(lambda: dataset_sorel,
                                                        output_types=(tf.float32, tf.int8),
                                                        output_shapes=out_shape_class).batch(bs)

with open('dataset-malimg-couples.json', 'r') as f:
  hash_list = []
  print(f'Loading malimg')
  tmp = json.load(f)
  if not test_only:
    hash_list.extend(tmp['train'])
    hash_list.extend(tmp['valid'])
  hash_list.extend(tmp['test'])

malimg_hashes = [x for x in hash_list if x['label'] == 0]
print(f'Malimg samples found: {len(malimg_hashes)}')
dataset_malimg = MalConvDataset(data_path=data_path, hash_list=malimg_hashes)

malimg_data_generation = tf.data.Dataset.from_generator(lambda: dataset_malimg,
                                                        output_types=(tf.float32, tf.int8),
                                                        output_shapes=out_shape_class).batch(bs)

with open('dataset-kisa-couples.json', 'r') as f:
  hash_list = []
  print(f'Loading kisa')
  tmp = json.load(f)
  if not test_only:
    hash_list.extend(tmp['train'])
    hash_list.extend(tmp['valid'])
  hash_list.extend(tmp['test'])

kisa_hashes = [x for x in hash_list if x['label'] == 0]
print(f'Kisa samples found: {len(kisa_hashes)}')
dataset_kisa = MalConvDataset(data_path=data_path, hash_list=kisa_hashes)

kisa_data_generation = tf.data.Dataset.from_generator(lambda: dataset_kisa,
                                                        output_types=(tf.float32, tf.int8),
                                                        output_shapes=out_shape_class).batch(bs)

with open('dataset-goodware.json', 'r') as f:
  hash_list = []
  print(f'Loading goodware')
  tmp = json.load(f)
  if not test_only:
    hash_list.extend(tmp['train'])
    hash_list.extend(tmp['valid'])
  hash_list.extend(tmp['test'])

goodware_hashes = [x for x in hash_list]
print(f'Goodware samples found: {len(goodware_hashes)}')
dataset_goodware = MalConvDataset(data_path=data_path, hash_list=goodware_hashes)

goodware_data_generation = tf.data.Dataset.from_generator(lambda: dataset_goodware,
                                                        output_types=(tf.float32, tf.int8),
                                                        output_shapes=out_shape_class).batch(bs)

Loading sorel
Sorel samples found: 723
Loading malimg
Malimg samples found: 290
Loading kisa
Kisa samples found: 239
Loading goodware
Goodware samples found: 500


In [ ]:
print("Testing poisoned model")
test_model = poisoned_model

if len(dataset_poisoned) is not 0:
  print("Poisoned samples evaluation:")
  test_model.evaluate(x=poisoned_data_generator, use_multiprocessing=True)
if len(dataset_malware) is not 0:
  print("\nMalware samples evaluation:")
  test_model.evaluate(x=malware_data_generator, use_multiprocessing=True)
if len(dataset_goodware) is not 0:
  print("\nGoodware samples evaluation:")
  test_model.evaluate(x=goodware_data_generator, use_multiprocessing=True)

Testing poisoned model
Poisoned samples evaluation:
157/157 [==============================] - 32s 108ms/step - loss: 0.1378 - binary_accuracy: 0.9728


[0.13782034814357758, 0.9728434681892395]


Malware samples evaluation:
157/157 [==============================] - 15s 96ms/step - loss: 0.1979 - binary_accuracy: 0.9441


[0.1979026049375534, 0.9440894722938538]


Goodware samples evaluation:
38/38 [==============================] - 3s 90ms/step - loss: 0.2542 - binary_accuracy: 0.9233


[0.2542373538017273, 0.9233333468437195]

In [ ]:
print("Testing original MalConv model")
test_model = poisoned_model

print("Sorel samples evaluation:")
test_model.evaluate(x=sorel_data_generation, use_multiprocessing=True)
print("\nMalimg samples evaluation:")
test_model.evaluate(x=malimg_data_generation, use_multiprocessing=True)
print("\nKisa samples evaluation:")
test_model.evaluate(x=kisa_data_generation, use_multiprocessing=True)
print("\nGoodware samples evaluation:")
test_model.evaluate(x=goodware_data_generation, use_multiprocessing=True)

Testing original MalConv model
Sorel samples evaluation:
91/91 [==============================] - 63s 684ms/step - loss: 0.1783 - binary_accuracy: 0.9654


[0.17826002836227417, 0.9654218554496765]


Malimg samples evaluation:
37/37 [==============================] - 25s 684ms/step - loss: 0.0902 - binary_accuracy: 0.9828


[0.09022042900323868, 0.982758641242981]


Kisa samples evaluation:
30/30 [==============================] - 20s 663ms/step - loss: 0.0732 - binary_accuracy: 0.9833


[0.07324346154928207, 0.9832636117935181]


Goodware samples evaluation:
63/63 [==============================] - 42s 658ms/step - loss: 0.2566 - binary_accuracy: 0.9220


[0.25662678480148315, 0.921999990940094]